PROJEKT R - CROBEX ANALIZA

Jupyter bilježnica koja sadrži skripte i algoritme korištene pri izradi projekta u sklopu kolegija Projekt R pod vodstvom mentora doc. dr. sc. Stjepana Begušića na Fakultetu elektrotehnike i računarstva.

Projekt se bavi analizom cjenovnih kretanja dionica na ZSE u usporedbi sa indexom CROBEX s obzirom na insertions/deletions.

Napomena: zbog količine skripti ne pokretati sve kodove odjednom

1. PRIKUPLJANJE i UREĐIVANJE PODATAKA

Prikupljanje informacija iz revizija o ulascima/izlascima dionica s CROBEX indeksa. 

In [ ]:
#skripte kojima smo pokušali skinuti podatke iz revizija, što smo ipak na kraju ručno odradili i pohranili u data.xlsx

#staviti kod ovdje:...
#Skripta web-data-extract.py
#Skripta automatic-link-getter.py ?
#Skripta link-parser.py ?


Skripta koja iz data.xlsx vraća sve tickere dionica pronađenih u stupcima isključeni(3), uključeni(4) i pohranja ih u tickers.txt.  
(ticker-xslx-extract.py)

In [ ]:
#Skripta ticker-xslx-extract.py
import os
import openpyxl

BASE_DIR = os.path.dirname(os.path.abspath(__file__))


EXCEL_FILE = os.path.join(BASE_DIR, "data.xlsx")      
OUTPUT_FILE = os.path.join(BASE_DIR, "tickers.txt")   


wb = openpyxl.load_workbook(EXCEL_FILE, data_only=True)
ws = wb["List1"]  # koristi List1

tickers = set()


def add_tickers_from_column(column_index, start_row=2, end_row=55):
    for row in range(start_row, end_row + 1):
        cell_value = ws.cell(row=row, column=column_index).value
        if not cell_value:
            continue
        parts = [t.strip().upper() for t in str(cell_value).split(";") if t.strip()]
        tickers.update(parts)


add_tickers_from_column(3)  #stupac isključeni
add_tickers_from_column(4)  #stupac uključeni


with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for ticker in sorted(tickers):
        f.write(ticker + "\n")

print(f"Izdvojeno je {len(tickers)} tickera.")
print(f"Spremljeno u datoteku: {OUTPUT_FILE}")

Prikupljanje povijesnih csv podataka o dionicama koje su nekad u periodu od 10.6.2010. do 22.9.2025. uključene/isključene s CROBEX-a.
(historical-data-script.py)

In [ ]:
#Skripta historical-data-script.py
import os
import requests

input_file = 'txt\\ticker-isin.txt'
output_dir = 'downloaded_xlsx'
start_date = '2010-01-01'
end_date = '2025-10-27'

os.makedirs(output_dir, exist_ok=True)

with open(input_file, 'r') as f:
    for line in f:
        line = line.strip()
        if not line: continue
        ticker, isin = line.split(",")
        url = f'https://rest.zse.hr/web/Bvt9fe2peQ7pwpyYqODM/security-history/XZAG/{isin}/{start_date}/{end_date}/xlsx'
        xlsx_path = os.path.join(output_dir, f'{ticker}_{isin}.xlsx')
        print(f'Dohvaćam {ticker} ({isin}) ...')
        resp = requests.get(url)
        with open(xlsx_path, 'wb') as outf:
            outf.write(resp.content)

Skripta koja spaja sve xlsx datoteke pojedinih dionica iz direktorija downloaded_xslx u jednu veliku xlsx datoteku sve_dionice_merged.xslx 
(xlsx-merge.py)

In [ ]:
#Skripta xlsx-merge.py
import os
import pandas as pd

input_dir = 'c:\\ferprogrami\\.vscode\\PROJEKT R\\downloaded_xlsx'
output_excel = 'sve_dionice_merged.xlsx'

xlsx_fajlovi = [f for f in os.listdir(input_dir) if f.endswith('.xlsx')]
if not xlsx_fajlovi:
    raise Exception("Nema .xlsx datoteka u folderu!")

with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
    sheet_unesen = False
    for xlsx_file in xlsx_fajlovi:
        path = os.path.join(input_dir, xlsx_file)
        ticker = xlsx_file.split('_')[0]
        sheet_name = ticker[:31]
        try:
            df = pd.read_excel(path)
            if not df.empty:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
                sheet_unesen = True
        except Exception as e:
            print(f"Preskačem {xlsx_file} zbog greške: {e}")

    if not sheet_unesen:
        raise Exception("Niti jedan sheet nije kreiran – provjeri ulazne fajlove!")

print("Sve spojeno, svaki sheet je ticker.")

Podskripta koja se "rješava" duplih dionica (dionice koje se u revizijama spominju kao ticker i  ticker-R-A) koja se nalazi u skriptama A_HRK_EURconv.py, B_dodavanjeiMicanjeRedaka.py i C_sinkronizacija.py. 

In [3]:
# === Postavke ===
#SRC = "sve_dionice_merged.xlsx"          # ulazna datoteka

# === Učitavanje i obrada ===
#xls = pd.ExcelFile(SRC)

# zadrži:
# - sve sheetove bez -R-A
# - sheetove s -R-A samo ako ne postoji njihova verzija bez nastavka

def filteredRAsheets(xls):
    all_sheets = xls.sheet_names
    filtered_sheets=[]
    izbaceni=[]
    for name in all_sheets:
        if name.endswith("-R-A"):
            if name.replace("-R-A", "") not in all_sheets:
                filtered_sheets.append(name)
            else:
                izbaceni.append(name)
        else:
            filtered_sheets.append(name)
    print("Izbaceno",len(izbaceni),"duplih sheetova")
    return filtered_sheets

Skripta koja za sve_dionice_merged.xslx konvertira sve vrijednosti iz HRK u EUR.            
(A_HRK_EURconv.py)

In [2]:
#Skripta A_HRK_EURconv.py
import pandas as pd
import xlsxwriter
# === Postavke ===
SRC = "sve_dionice_merged.xlsx"          # ulazna datoteka
DST = "sve_dionice_merged_EUR.xlsx"      # izlazna datoteka
FX = 7.53450                             # fiksni tečaj HRK → EUR

PRICE_COLS = [
    "Open Price", "High Price", "Low Price",
    "Last Price", "VWAP Price", "Prev Close Price"
]
TURNOVER_COL = "Turnover"
PRICE_CCY_COL = "Price Currency"
TURNOVER_CCY_COL = "Turnover Currency"

# === Učitavanje i obrada ===
xls = pd.ExcelFile(SRC)
summary_rows = []

filtered_sheets=filteredRAsheets(xls)

with pd.ExcelWriter(DST, engine="xlsxwriter") as writer:
    for sheet in filtered_sheets:
        df = xls.parse(sheet)
        df.columns = [str(c).strip() for c in df.columns]

        # maske gdje su valute HRK
        price_mask = df[PRICE_CCY_COL].eq("HRK") if PRICE_CCY_COL in df.columns else pd.Series([False]*len(df))
        turnover_mask = df[TURNOVER_CCY_COL].eq("HRK") if TURNOVER_CCY_COL in df.columns else pd.Series([False]*len(df))

        # konverzija cijena
        for col in PRICE_COLS:
            if col in df.columns:
                df.loc[price_mask, col] = df.loc[price_mask, col] / FX
                df[col] = df[col].round(4)
        if PRICE_CCY_COL in df.columns:
            df.loc[price_mask, PRICE_CCY_COL] = "EUR"

        # konverzija prometa
        if TURNOVER_COL in df.columns:
            df.loc[turnover_mask, TURNOVER_COL] = df.loc[turnover_mask, TURNOVER_COL] / FX
            df[TURNOVER_COL] = df[TURNOVER_COL].round(2)
        if TURNOVER_CCY_COL in df.columns:
            df.loc[turnover_mask, TURNOVER_CCY_COL] = "EUR"

        # spremi konvertirani sheet
        df.to_excel(writer, sheet_name=sheet, index=False)

        summary_rows.append({
            "Sheet": sheet,
            "Rows": len(df),
            "HRK→EUR (price rows)": int(price_mask.sum()),
            "HRK→EUR (turnover rows)": int(turnover_mask.sum())
        })

# === Sažetak ===
summary = pd.DataFrame(summary_rows).sort_values("Sheet")
print("\n=== HRK → EUR konverzija: sažetak ===")
print(summary.to_string(index=False))
print(f"\nKonvertirana datoteka spremljena kao: {DST}")

Izbaceno 26 duplih sheetova

=== HRK → EUR konverzija: sažetak ===
   Sheet  Rows  HRK→EUR (price rows)  HRK→EUR (turnover rows)
    ADPL  3845                  3178                     3178
    ARNT  3234                  2646                     2646
    ATPL  3649                  3190                     3190
    AUHR  1968                  1650                     1650
    BLJE  1885                  1885                     1885
BLSC-R-A   342                   342                      342
    CKML  1888                  1413                     1413
    DDJH  3341                  3041                     3041
DIOK-R-A   969                   969                      969
    DLKV  3677                  3068                     3068
HIMR-R-A  1516                  1409                     1409
    HMST  1091                  1091                     1091
     HPB  2017                  1392                     1392
    HUPZ  1647                  1647                     1647
   

Skripta koja za sve datume za koje se nekom dionicom trgovalo drugim dionicama kojima se nije dodaje redke u xlsx no pritom za volume i turnover stavlja 0. Također miče block i OTC trgovanja.                     
(B_dodavanjeiMicanjeRedaka.py)

In [4]:
#Skripta B_dodavanjeiMicanjeRedaka.py
import pandas as pd

SRC = "sve_dionice_merged_EUR.xlsx"
DST = "sve_dionice_merged_EUR_filled.xlsx"

PRICE_COLS = ["Open Price","High Price","Low Price","Last Price","VWAP Price","Prev Close Price"]
ZERO_COLS  = ["Volume","Num Trades","Turnover"]
META_COLS  = ["MIC","Symbol","ISIN","Price Currency"]
DATE_COL   = "Date"
MODEL_COL  = "Trading Model"
FILTER_R_A = True

xls = pd.ExcelFile(SRC)
sheet_names = xls.sheet_names


if FILTER_R_A:
    filtered = []
    for n in sheet_names:
        if n.endswith("-R-A"):
            if n.replace("-R-A","") not in sheet_names:
                filtered.append(n)
        else:
            filtered.append(n)
    sheet_names = filteredRAsheets(xls)

# Unija svih datuma
all_dates = set()
for sh in sheet_names:
    df = xls.parse(sh)
    df.columns = [str(c).strip() for c in df.columns]
    if DATE_COL not in df.columns:
        continue
    dates = pd.to_datetime(df[DATE_COL], errors="coerce").dt.normalize().dropna()
    all_dates.update(dates.tolist())
global_dates = pd.DatetimeIndex(sorted(all_dates))

with pd.ExcelWriter(DST, engine="xlsxwriter") as writer:
    for sh in sheet_names:
        df = xls.parse(sh)
        df.columns = [str(c).strip() for c in df.columns]
        if DATE_COL not in df.columns:
            df.to_excel(writer, sheet_name=sh, index=False); continue

        # Normaliziraj datum
        df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce").dt.normalize()
        df = df.dropna(subset=[DATE_COL])

        # Makni BLOCK/OTC redke (ako stupac postoji)
        if MODEL_COL in df.columns:
            mask_bad = df[MODEL_COL].astype(str).str.upper().str.strip().isin({"BLOCK","OTC"})
            df = df[~mask_bad]

        # Ako je sve ispalo, samo snimi prazan rezultat
        if df.empty:
            df.to_excel(writer, sheet_name=sh, index=False); continue

        df = df.sort_values(DATE_COL).groupby(DATE_COL, as_index=False).last().set_index(DATE_COL)

        active_start, active_end = df.index.min(), df.index.max()
        wanted_idx = global_dates[(global_dates >= active_start) & (global_dates <= active_end)]

        re = df.reindex(wanted_idx)
        new_rows_mask = re.index.difference(df.index)
        new_rows_mask = re.index.isin(new_rows_mask)

        # FFill cijene i meta-info
        for col in PRICE_COLS + META_COLS:
            if col in re.columns:
                re[col] = re[col].ffill()

        # Nove dane -> nule za volumene/trgovanja/promet
        for col in ZERO_COLS:
            if col in re.columns:
                re.loc[new_rows_mask, col] = 0

        re = re.reset_index().rename(columns={"index": DATE_COL})
        re.to_excel(writer, sheet_name=sh, index=False)

print(f"Gotovo Spremio sam: {DST}")

Izbaceno 0 duplih sheetova
Gotovo Spremio sam: sve_dionice_merged_EUR_filled.xlsx


Skripta koja grafički prikaže sinkronizirane podatke svih dionica za neku od metrika (Volume, Last Price, Turnover...)  
(C_sinkronizacija.py)

In [ ]:
#Skripta C_sinkronizacija.py
import pandas as pd
file_path = "..\sve_dionice_merged_EUR_filled.xlsx"

xls = pd.ExcelFile(file_path)

print("Sheetovi pronađeni u datoteci:")

# zadrži:
# - sve sheetove bez -R-A
# - sheetove s -R-A samo ako ne postoji njihova verzija bez nastavka
all_sheets = xls.sheet_names
base_names = set(s.replace("-R-A", "") for s in all_sheets)
filtered_sheets = []
izbaceni=[]
for name in all_sheets:
    if name.endswith("-R-A"):
        if name.replace("-R-A", "") not in all_sheets:
            filtered_sheets.append(name)
        else:
            izbaceni.append(name)
    else:
        filtered_sheets.append(name)

# 2) Odaberi metriku i agregaciju (za Volume/Num Trades često je "sum", za cijene "last")
metric = "Last Price"  # npr. "Volume", "Num Trades", "Last Price", "VWAP Price", "Turnover"
default_agg = "last"
per_metric_agg = {"Volume": "sum", "Num Trades": "sum"}
agg = per_metric_agg.get(metric, default_agg)

# 3) Inkrementalno gradi pivot (outer join po datumu)
pivot = pd.DataFrame()

for sheet in filtered_sheets:
    df = xls.parse(sheet)
    df.columns = [str(c).strip() for c in df.columns]
    if "Date" not in df.columns or metric not in df.columns:
        continue

    # Parsiraj datum na date (bez vremena) i ukloni loše retke
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce").dt.date
    df = df.dropna(subset=["Date"])

    # Agregiraj na dnevnu razinu po metrikama
    if agg == "last":
        daily = df.groupby("Date", as_index=True)[metric].last()
    elif agg == "sum":
        daily = df.groupby("Date", as_index=True)[metric].sum(min_count=1)
    elif agg == "mean":
        daily = df.groupby("Date", as_index=True)[metric].mean()
    elif agg == "max":
        daily = df.groupby("Date", as_index=True)[metric].max()
    elif agg == "min":
        daily = df.groupby("Date", as_index=True)[metric].min()
    else:
        # fallback na "last" ako je zadano nešto drugo
        daily = df.groupby("Date", as_index=True)[metric].last()

    daily = daily.rename(sheet).to_frame()

    # outer join da se širi skup datuma i stupaca
    pivot = daily if pivot.empty else pivot.join(daily, how="outer")

# 4) Uredi indeks (datumi) i opcijski spremi
pivot = pivot.sort_index(ascending=False)  # padajući datumi
# pivot.to_excel("pivot_output.xlsx", merge_cells=False)
# pivot.to_csv("pivot_output.csv")

print(pivot.shape)
print(pivot.head(10))

import os
import matplotlib.pyplot as plt

# 1) SVE DIONICE NA JEDNOM GRAFU (oprez: može biti nepregledno)
pivot.sort_index().plot(legend=False)
plt.title("Vrijednosti kroz vrijeme - sve dionice")
plt.xlabel("Datum")
plt.ylabel("Vrijednost")
plt.tight_layout()
plt.show()

# 2) PO JEDAN GRAF PO DIONICI (preporučeno)
out_dir = "plots_by_ticker"
os.makedirs(out_dir, exist_ok=True)

for col in pivot.columns:
    s = pivot[col].dropna().sort_index()
    if s.empty:
        continue
    # (opcija) downsample na tjedno da bude preglednije:
    # s = s.resample("W").median()
    plt.figure()
    s.plot()
    plt.title(f"{col} — vrijednost kroz vrijeme")
    plt.xlabel("Datum")
    plt.ylabel("Vrijednost")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"{col}.png"), dpi=150)
    plt.close()

2. ANALIZA PODATAKA